# High-Level Structures

We have seen before how to deal with `tq.Objective`s (functions of abstract expectation values). In the following we learn about two supported high-level structures that can come in Handy in some situations.

- `tq.QTensor`: tensors of `tq.Objectives` with similar behaviour and usage (basically a numpy.ndarray with `tq.Objective` as datatype + some convenience)
- `tq.BraKet`: general transition elements $\langle \Psi_1 \rvert H \lvert \Psi_0 \rangle$ automatically decomposed into the `tq.Objective` format.

In [1]:
import tequila as tq
import numpy

## BraKets

The usage is straightforward. One potential stepstone: $\langle \Psi_1 \rvert H \lvert \Psi_0 \rangle$ are complex values. In order to stay consistent with the system we need all elements to evaluate to real values, so the real and imaginary part of the BraKet are treated separately.

In [30]:
U0 = tq.gates.Ry("a",0)
U1 = tq.gates.H(0) + tq.gates.CNOT(0,1)
H = tq.paulis.X(0)*tq.paulis.Z(1)

real, imag = tq.BraKet(bra=U0, ket=U1, operator=H)

In [31]:
print(real)

Objective with 1 unique expectation values
total measurements = 1
variables          = [a]
types              = not compiled


In [32]:
# let's see how the circuits changed
for ev in real.get_expectationvalues():
    circuit = ev.U
    print("circuit on {} qubits".format(circuit.n_qubits))
    print(circuit)

circuit on 3 qubits
circuit: 
H(target=(2,))
X(target=(2,))
Ry(target=(0,), control=(2,), parameter=a)
X(target=(2,))
H(target=(0,), control=(2,))
X(target=(1,), control=(0, 2))
X(target=(0,), control=(2,))
Z(target=(1,), control=(2,))



## QTensors

We can initialize QTensors like numpy.ndarrays. Important to note: It is not necessary to evaluate the expectation values. Here is a small example that introduces a small matrix and a vector consisting of `tq.Objective`s (some parametrized, some not) and real numbers.

In [33]:
# initializing some structures
U0 = tq.gates.Rp(paulistring="Y(0)",angle="a")
U1 = tq.gates.Rp(paulistring="Y(0)",angle=1.0)
H0 = tq.paulis.X(0)

E0 = tq.ExpectationValue(H=H0, U=U0)
E1 = tq.ExpectationValue(H=H0, U=U1)
F0, F1 = tq.BraKet(bra=U0, ket=U1)

# initialize the QTensors
M = tq.QTensor([E0,E1,F0,1.0], shape=[2,2])
V = tq.QTensor([1.0,F1], shape=[2,])

# operate on them
W = M.dot(V)

Let's see what we got 

In [34]:
print("abstract structure")
print(W)

# proceed like before with standard scalar objectives
callable_W = tq.compile(W)

print("\n\ncallable structure")
print(callable_W)

evaluated = callable_W(variables={"a":1.0})
print("\n\nevaluated structure")
print(evaluated)

abstract structure
QTensor of shape (2,) with 4 unique expectation values
total measurements = 4
variables          = [a]
types              = not compiled


callable structure
QTensor of shape (2,) with 5 unique expectation values
total measurements = 5
variables          = [a]
types              = [<class 'tequila.simulators.simulator_qulacs.BackendExpectationValueQulacs'>]


evaluated structure
[0.84147098 1.        ]


## Further reading

### Code

- Example of a Quantum Krylov Method (QKM) by Francesco Scala  
https://github.com/tequilahub/tequila-tutorials/blob/main/KrylovTutorial.ipynb

- More information on the QTensor class by Gaurav Saxena  
https://github.com/tequilahub/tequila-tutorials/blob/main/QTensor.ipynb

- More information on the BraKet feature by Francesco Scala  
https://github.com/tequilahub/tequila-tutorials/blob/main/BraketTutorial.ipynb

### Literature

Some methods, where the structures above come in handy:

- https://arxiv.org/abs/2302.10660 (using the tq implementation, explicit code example in appendix, accelerated [code](https://github.com/tequilahub/compact-bases))
- https://arxiv.org/abs/1911.05163 (one of the first quantum krylov approaches in QChem, [code](https://github.com/evangelistalab/qforte))
- https://arxiv.org/abs/1909.09114 (non-variational quantum eigensolver, no code)